# Parallelizing Image Convolution

## Learning Goals

By the end of this tutorial, you will be able to:

- Employ three parallelization libraries to speed up a serial process.
- Calculate the speedup of the different approaches shown.
- Evaluate which library is suited to your task.

## Introduction

This notebook shows how to speed up an image convolution task using these three libraries:

* Ray: an open-source unified compute framework that makes it easy to scale AI and Python workloads.
* Multiprocessing: part of the standard library; supports spawning processes using an API similar to the threading module; offers both local and remote concurrency, effectively side-stepping the Global Interpreter Lock by using subprocesses instead of threads.
* Dask: developed to natively scale computational packages like numpy, pandas and scikit-learn, and the surrounding ecosystem, to multi-core machines and distributed clusters when datasets exceed memory.

## Imports

* _multiprocessing.Pool_ for multiprocessing using the standard library
* _time_ for timing the processes
* _dask.distributed.Client_ for making a local Dask cluster
* _numpy_ and _scipy.signal_ for numerical work
* _psutil_ for finding the available processors on your machine
* _ray_ for scaling up Python tasks

In [1]:
from multiprocessing import Pool
import time

from dask.distributed import Client
import numpy as np
import psutil
import scipy.signal
import ray

2024-08-29 19:06:10,388	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Find the cpus available

Find and print the number of cpus
(taken from https://towardsdatascience.com/10x-faster-parallel-python-without-python-multiprocessing-e5017c93cce1)

In [2]:
num_cpus = psutil.cpu_count(logical=True)
print(num_cpus)

4


## Process serially using a conventional loop

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result.

In [3]:
def fconv(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In [4]:
filters = [np.random.normal(size=(4, 4)) for _ in range(num_cpus)]

Process 100 iterations serially, then extrapolate to num_cpus*100

In [5]:
start = time.time()
num_iter = 100
image = np.zeros((3000, 3000))
for i in range(num_iter):
    result = fconv(image, filters[i % num_cpus])
duration_conv = time.time() - start
print("(scaled) conventional duration for {:d} iterations = {:.1f} seconds"
      .format(num_cpus*num_iter, duration_conv*num_cpus))

(scaled) conventional duration for 400 iterations = 150.3 seconds


## Process in parallel using Ray

[Documentation for ray](https://docs.ray.io/en/latest/)

The warning raised by `ray.init` only affects shared object usage, which is not an issue for this tutorial. It may harm performance in other scenarios.

In [6]:
ray.init(num_cpus=num_cpus)

2024-08-29 19:06:49,923	INFO worker.py:1783 -- Started a local Ray instance.


Python version:,3.11.9
Ray version:,2.35.0


Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. To use Ray, we decorate the function that is doing the work.

In [7]:
@ray.remote
def fray(image, random_filter):
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

In the following loop, `ray.put` places the image into shared memory. The call to `ray.get` retrieves the result.

In [8]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    image_id = ray.put(image)
    ray.get([fray.remote(image_id, filters[i]) for i in range(num_cpus)])
duration_ray = time.time() - start
print("Ray duration = {:.1f}, speedup = {:.2f}"
      .format(duration_ray, duration_conv*num_cpus / duration_ray))

Ray duration = 86.3, speedup = 1.74


In [9]:
ray.shutdown()

## Process in parallel using multiprocessing

[Documentation for multiprocessing](https://docs.python.org/3/library/multiprocessing.html)

Use `scipy.signal` to convolve two 2-dimensional arrays and return a 5x5 downsampled result. The call to the function has a slightly different form than that for the serial loop.

In [10]:
def fmp(args):
    image, random_filter = args
    return scipy.signal.convolve2d(image, random_filter)[::5, ::5]

Use a multiprocessing pool with the number of cpus we found earlier.

In [11]:
pool = Pool(num_cpus)

Using `pool.map` is the closest analog in multiprocessing to the Ray API.

In [12]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    pool.map(fmp, zip(num_cpus * [image], filters))
duration_mp = time.time() - start
print("Multiprocessing duration = {:.1f}, speedup = {:.2f}"
      .format(duration_mp, duration_conv*num_cpus / duration_mp))

Multiprocessing duration = 143.8, speedup = 1.05


## Process using Dask

[Documentation for Dask](https://www.dask.org/get-started)

Define a Dask distributed client with number of workers set to the number of cpus we found earlier, and with one thread per worker.

In [13]:
client = Client(n_workers=num_cpus, threads_per_worker=1)

In [14]:
print(client)

<Client: 'tcp://127.0.0.1:35229' processes=4 threads=4, memory=15.61 GiB>


Dask recommends scattering the large inputs across the workers, though this makes little difference in execution time.

In [15]:
start = time.time()
image = np.zeros((3000, 3000))
for _ in range(100):
    for j in range(num_cpus):
        big_future = client.scatter((image, filters[j % num_cpus]))
        future = client.submit(fconv, big_future)
duration_dask = time.time() - start
print("Dask duration = {:.1f}, speedup = {:.2f}"
      .format(duration_dask, duration_conv*num_cpus / duration_dask))

2024-08-29 19:10:44,572 - distributed.worker - ERROR - Compute Failed
Key:       fconv-854e8812be5cdc2168ba1cc409c41afb
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.62715777, -1.46872263,  1.09426539, -0.31068653],
       [-0.03605778,  0.71007963, -1.15404467, -0.22782133],
       [-0.00879041, -0.21080391,  0.88728598, -1.7043    ],
       [-1.09334027,  0.66184451, -1.60529596,  0.00970918]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:10:44,665 - distributed.worker - ERROR - Compute Failed
Key:       fconv-a64750e98c89a4eec128278e4dfb5630
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 19:10:44,855 - distributed.worker - ERROR - Compute Failed
Key:       fconv-aa04d73430745a43d64e94d34adcba91
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 7.38193801e-01, -7.28836328e-01, -3.36977404e-01,
        -1.73513253e+00],
       [-4.36515552e-01, -4.66498521e-01,  7.51503549e-01,
         7.90187445e-04],
       [ 1.87616436e-01, -1.26251231e-01, -9.09359160e-01,
        -1.47630629e+00],
       [-1.09520064e+00,  2.42220818e-01, -8.96537272e-01,
         1.28067235e+00]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:10:44,956 - distributed.worker - ERROR - Compute Failed
Key:       fconv-69cb8b50ed8d0a76928b2b1a802e855c
Sta

2024-08-29 19:10:45,151 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2962c88e0c26477436343746bc6265a5
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.53719254, -1.30241094,  0.86799871,  0.32259247],
       [ 0.74037679, -0.43049249, -2.64861704,  0.55766254],
       [ 0.21712709,  0.65725021, -1.00967844,  1.36509094],
       [ 0.79258618, -1.56538813,  1.56547117, -0.69933211]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:10:45,242 - distributed.worker - ERROR - Compute Failed
Key:       fconv-300aaca1a2268714a96332b7e3939d88
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 19:10:45,420 - distributed.worker - ERROR - Compute Failed
Key:       fconv-9647ac94a88393366383c77e4af00d01
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.59158641,  0.39068371,  0.07113865,  0.9599546 ],
       [ 1.01814735,  1.05251596,  1.13913608, -1.10717879],
       [-0.42200201,  0.49037922, -1.07326311, -1.40643818],
       [-0.51460145,  0.22592446,  0.14521425, -0.56053994]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:10:45,516 - distributed.worker - ERROR - Compute Failed
Key:       fconv-e209007c2af8f43e240d51339756c5ae
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 19:10:45,705 - distributed.worker - ERROR - Compute Failed
Key:       fconv-8aa216515db217eb13737cb37a17f6f7
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.62715777, -1.46872263,  1.09426539, -0.31068653],
       [-0.03605778,  0.71007963, -1.15404467, -0.22782133],
       [-0.00879041, -0.21080391,  0.88728598, -1.7043    ],
       [-1.09334027,  0.66184451, -1.60529596,  0.00970918]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:10:45,799 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2f5b614c2ce8ea03cefb524a76a2b5ef
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

...

2024-08-29 19:11:21,008 - distributed.worker - ERROR - Compute Failed
Key:       fconv-2f34bcdf11064338207a16ede4abad5f
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.53719254, -1.30241094,  0.86799871,  0.32259247],
       [ 0.74037679, -0.43049249, -2.64861704,  0.55766254],
       [ 0.21712709,  0.65725021, -1.00967844,  1.36509094],
       [ 0.79258618, -1.56538813,  1.56547117, -0.69933211]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:11:21,094 - distributed.worker - ERROR - Compute Failed
Key:       fconv-07860c0445753b0e5ba7b1d4e15b0b42
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 19:11:21,280 - distributed.worker - ERROR - Compute Failed
Key:       fconv-7444670180cff5bd6800aa1b4eda478e
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 0.59158641,  0.39068371,  0.07113865,  0.9599546 ],
       [ 1.01814735,  1.05251596,  1.13913608, -1.10717879],
       [-0.42200201,  0.49037922, -1.07326311, -1.40643818],
       [-0.51460145,  0.22592446,  0.14521425, -0.56053994]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:11:21,377 - distributed.worker - ERROR - Compute Failed
Key:       fconv-108fb4e0ac4fb405c0db734a882937fd
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

2024-08-29 19:11:21,565 - distributed.worker - ERROR - Compute Failed
Key:       fconv-bff51a617793742f801f0c2a6362f0fe
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[-1.62715777, -1.46872263,  1.09426539, -0.31068653],
       [-0.03605778,  0.71007963, -1.15404467, -0.22782133],
       [-0.00879041, -0.21080391,  0.88728598, -1.7043    ],
       [-1.09334027,  0.66184451, -1.60529596,  0.00970918]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:11:21,657 - distributed.worker - ERROR - Compute Failed
Key:       fconv-aff5cfd9b6a1f5150186d0baa003f3f3
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0.

Dask duration = 38.0, speedup = 3.96


2024-08-29 19:11:21,850 - distributed.worker - ERROR - Compute Failed
Key:       fconv-f2c79159be538c1d3067931e45b56a60
State:     executing
Function:  fconv
args:      ((array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]]), array([[ 7.38193801e-01, -7.28836328e-01, -3.36977404e-01,
        -1.73513253e+00],
       [-4.36515552e-01, -4.66498521e-01,  7.51503549e-01,
         7.90187445e-04],
       [ 1.87616436e-01, -1.26251231e-01, -9.09359160e-01,
        -1.47630629e+00],
       [-1.09520064e+00,  2.42220818e-01, -8.96537272e-01,
         1.28067235e+00]])))
kwargs:    {}
Exception: 'TypeError("fconv() missing 1 required positional argument: \'random_filter\'")'
Traceback: ''

2024-08-29 19:11:21,941 - distributed.worker - ERROR - Compute Failed
Key:       fconv-c4b799a39406e966cbd740a2a98e270d
Sta

In [16]:
client.close()

## Conclusions

* Ray is the most effective at speeding up the convolution workload by fully utilizing all available processes
* Multiprocessing is second in effectiveness
* Dask delivers the least speedup; perhaps due to having only six processes on the dask.distributed client

## About this notebook

This notebook was developed by David Shupe (shupe@ipac.caltech.edu) in conjunction with Jessica Krick and the IRSA Science Platform team at IPAC.

## Citations

If you use these software packages in your work, please use the following citations:

* Dask: Dask Development Team (2016). Dask: Library for dynamic task scheduling. URL https://dask.org
* Ray: The Ray Development Team. URL https://docs.ray.io